In [ ]:
!pip install datasets torch pandas evaluate rouge_score sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f39b896dffd97897bd326bd8457ff27605b14a95f9082b753290bb606868cf22
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate
from google.colab import drive
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, BartForConditionalGeneration
from tqdm import tqdm

import pandas as pd
import torch
import random

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
test_df = pd.DataFrame()

i = 50
for iteration in range(i,i+10):
  test_dataset_path = f"./drive/MyDrive/Submission/BE Project Group No 31/Data/test_data{iteration}.csv"
  test_df = pd.concat([test_df, pd.read_csv(test_dataset_path)])

# scillm/scientific_papers-archive
i = 121
for iteration in range(i,i+10):
  test_dataset_path = f"./drive/MyDrive/Submission/BE Project Group No 31/Data/test_data{iteration}.csv"
  test_df = pd.concat([test_df, pd.read_csv(test_dataset_path)])


test_dataset = Dataset.from_pandas( test_df )
papers_dataset = DatasetDict({"test":test_dataset})

In [ ]:
papers_dataset

DatasetDict({
    test: Dataset({
        features: ['article', 'abstract', '__index_level_0__'],
        num_rows: 1050
    })
})

In [ ]:
model_path = "./drive/MyDrive/Submission/BE Project Group No 31/Model/iter_trained_model"
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn",device=device)
model = BartForConditionalGeneration.from_pretrained(model_path).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=2, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

### Rouge Score

In [ ]:
rouge_metric = evaluate.load("rouge")

rouge_score = calculate_metric_on_test_ds(papers_dataset['test'], rouge_metric, model, tokenizer, column_text = 'article', column_summary='abstract', batch_size=8)

100%|██████████| 132/132 [22:46<00:00, 10.35s/it]


In [ ]:
pd.DataFrame(rouge_score,index=["Trained Model"])

,rouge1,rouge2,rougeL,rougeLsum
Trained Model,0.01478,0.000429,0.014318,0.014653


### Bleu Score

In [ ]:
bleu_metric = evaluate.load("sacrebleu")

bleu_score = calculate_metric_on_test_ds(papers_dataset['test'], bleu_metric, model, tokenizer, column_text = 'article', column_summary='abstract', batch_size=8)

100%|██████████| 132/132 [22:46<00:00, 10.35s/it]


In [ ]:
print("Bleu Score : ",bleu_score["score"])

Bleu Score :  0.08355345747385627
